
# Arguments

In [1]:
# vgg16-ft-pv-self-eb,ap,lm
num_of_epochs = 25
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

model_save_path = 'model-warmup.h5'

checkpoint_path = 'checkpoints-warmup.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# Print class names

In [2]:
# Get classes
import os
import re

class_names = []
class_paths_plant_village = []
class_paths_plant_self = []
class_paths_plant_validation = []

for path in dataset_paths:
    classes = os.listdir(path)
    for i in classes:
        if(re.search('___', i)):
            if(i not in class_names):
                class_names.append(i)
            if(re.search('plantvillage', path)):
                class_paths_plant_village.append(path + '/' + i)
            elif(re.search('self', path)):
                class_paths_plant_self.append(path + '/' + i)
            else:
                class_paths_plant_validation.append(path + '/' + i)

print(class_names)   
print()
print('PlantVillage')
print(class_paths_plant_village) 
print(len(class_paths_plant_village))
print()
print('Self')
print(class_paths_plant_self) 
print(len(class_paths_plant_self))
print()
print('Validation')
print(class_paths_plant_validation) 
print(len(class_paths_plant_validation))

['___Yellow_Leaf_Curl_Virus', '___Late_blight', '___Early_blight', '___Two-spotted_spider_mite', '___Septoria_leaf_spot', '___Bacterial_spot', '___healthy', '___Target_Spot', '___Mosaic_Virus', '___Leaf_Mold', '___Leaf_miner', '___Appids']

PlantVillage
['../datasets/plantvillage/___Yellow_Leaf_Curl_Virus', '../datasets/plantvillage/___Late_blight', '../datasets/plantvillage/___Early_blight', '../datasets/plantvillage/___Two-spotted_spider_mite', '../datasets/plantvillage/___Septoria_leaf_spot', '../datasets/plantvillage/___Bacterial_spot', '../datasets/plantvillage/___healthy', '../datasets/plantvillage/___Target_Spot', '../datasets/plantvillage/___Mosaic_Virus', '../datasets/plantvillage/___Leaf_Mold']
10

Self
['../datasets/self/___Leaf_miner', '../datasets/self/___Early_blight', '../datasets/self/___Appids']
3

Validation
['../datasets/validation/___Leaf_miner', '../datasets/validation/___Yellow_Leaf_Curl_Virus', '../datasets/validation/___Late_blight', '../datasets/validation/___E

# Select training classes

early blight in plantVillage dataset replaced by self one

In [3]:
class_paths_training = ['../datasets/self/___Early_blight', '../datasets/self/___Appids', '../datasets/self/___Leaf_miner']

# Load dataset

In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
image_paths = []
print("[INFO] loading images...")
for path in class_paths_training:
    image_paths.extend(list(paths.list_images(path)))
    
# Get unique classnames

class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
class_names = [str(x) for x in np.unique(class_names)]

# Initial image preprocessing
aap = AspectAwarePreprocessor(input_width, input_height)
iap= ImageToArrayPreprocessor()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(image_paths,verbose=500)
data = data.astype("float") / 255.0


# train test split
(train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

# convert the labels from integers to vectors
train_y = LabelBinarizer().fit_transform(train_y)
test_y = LabelBinarizer().fit_transform(test_y)



Using TensorFlow backend.


[INFO] loading images...
[INFO]: Processed 500/4374
[INFO]: Processed 1000/4374
[INFO]: Processed 1500/4374
[INFO]: Processed 2000/4374
[INFO]: Processed 2500/4374
[INFO]: Processed 3000/4374
[INFO]: Processed 3500/4374
[INFO]: Processed 4000/4374



# Load model

In [5]:
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model
from keras.layers import Input



# Load pretrianed VGG model with FC layers removed
# explicitly deﬁne the input_tensor to be 224×224×3 pixels
base_model = VGG16(weights='imagenet',include_top=False,
                  input_tensor=Input(shape = (input_width,input_height, input_depth)))

# plot_save_path = 'diagram-base-vgg16.png'
# plot_model(base_model, to_file=model_diagram_path, show_shapes=True)

# Custom FC layer

In [6]:
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

head_model = base_model.output
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256,activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
# Add a softmaxc layer
head_model = Dense(len(class_names),activation='softmax')(head_model)

# plot_save_path = 'custom-head.png'
# plot_model(head_model, to_file=model_diagram_path, show_shapes=True)

# Attach custom head to model

In [7]:
from keras.models import Model


# place the head FC model on top of the base model 
model = Model(inputs=base_model.input, outputs = head_model)

# freeze weights in base model
for layer in base_model.layers:
    layer.trainable = False

# plot_save_path = 'diagram-vgg16-head-attached.png'
# plot_model(model, to_file=model_diagram_path, show_shapes=True)

# Freeze base model

In [8]:
# traverse all layers and freeze the weight of the corresponding layer
for layer in base_model.layers:
    layer.trainable = False

# Compile model

In [9]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.001, momentum=0.09)

model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# Since we only train the new fully connected layer, we do a few iterations

[INFO] re-compiling model ...


# Checkpoints

In [10]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [11]:
#load checkpoints if existing

import os

epochs_done = 0
num_of_epochs = 10

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)
    num_of_epochs = num_of_epochs - epochs_done


# Training monitor

In [12]:
from utils.callbacks.training_monitor import TrainingMonitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'
start_epoch = 0

last_epoch = None
last_improved_epoch = None

if(os.path.exists(values_path)):
    with open(values_path) as file_content:
        data = json.load(file_content)
        start_epoch = len(data['val_accuracy']) - (last_epoch - last_improved_epoch)



callbacks.append(TrainingMonitor(fig_path, json_path, start_epoch))

# Warm up head

In [13]:
num_of_epochs = 50

model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

model.save(model_save_path)

Epoch 1/50
102/102 [==============================] - 56s 545ms/step - loss: 0.9957 - accuracy: 0.5243 - val_loss: 0.7400 - val_accuracy: 0.7102

Epoch 00001: val_loss improved from inf to 0.74003, saving model to checkpoints-warmup.hdf5
Epoch 2/50
102/102 [==============================] - 50s 491ms/step - loss: 0.7689 - accuracy: 0.6733 - val_loss: 0.5781 - val_accuracy: 0.7925

Epoch 00002: val_loss improved from 0.74003 to 0.57809, saving model to checkpoints-warmup.hdf5
Epoch 3/50
102/102 [==============================] - 50s 491ms/step - loss: 0.6355 - accuracy: 0.7442 - val_loss: 0.4914 - val_accuracy: 0.8336

Epoch 00003: val_loss improved from 0.57809 to 0.49140, saving model to checkpoints-warmup.hdf5
Epoch 4/50
102/102 [==============================] - 50s 493ms/step - loss: 0.5551 - accuracy: 0.7799 - val_loss: 0.3971 - val_accuracy: 0.8775

Epoch 00004: val_loss improved from 0.49140 to 0.39710, saving model to checkpoints-warmup.hdf5
Epoch 5/50
102/102 [================

# Evaluate with 20% test set from same dataset

In [14]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=32)
print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),target_names=class_names))

[INFO] evaluating after initialization...
                 precision    recall  f1-score   support

      ___Appids       0.93      0.99      0.96       243
___Early_blight       0.98      0.91      0.94       394
  ___Leaf_miner       0.93      0.96      0.95       457

    avg / total       0.95      0.95      0.95      1094

